# CASE STUDY 1 - GROUP 2
### Members
- Bautista, Millette
- Dano, Jomari
- Narzoles, John Peter
- Roranes, Raven Rain
- Siaotong, Danica

---

### SETUP

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


base_url = "https://philatlas.com/"

---

### FUNCTIONS

In [2]:
# url = webpage
# element = target element
# element_class = class of target element
def get_specific_element(url, element, element_class):
    page = requests.get(url)
    if page.status_code != 200:
        return "Error on accessing Webpage"
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup.find(element, class_=element_class)

---

### Output 1: Island Group, Region

In [48]:
# VARIABLES

# OUTPUT 1
island_groups = []
regions = []
url = f"{base_url}island-groups.html"

# OUTPUT 2
REGIONS = {}

In [49]:
right_table = get_specific_element(url, 'table', 'generic-table')

for row in right_table.find_all('tr'):
    #kinuha island group kasi di sya kasama sa td

    island = row.find_all('th', scope = 'row')
    if len(island) > 0:
    #     #finind ko yung laman ul para ilabas lahat ng mga list na nandun.
        _region = row.find('ul').text
        island_groups.append(island[0].find(text=True))
        regions.append(_region+' ')


        # SETUP FOR OUTPUT 2
        links = row.find('ul')
        for _url in links.find_all('a'):
            _region_url = base_url + _url.get('href')
            _txt = _url.get_text()
            _region_name = _txt.split(' – ')[0].split(' ')[0]
            REGIONS[_region_name] = _region_url

In [50]:
df_output1 = pd.DataFrame(island_groups, columns=['Island group'])
df_output1['Component regions'] = regions

In [6]:
df_output1.to_csv('../Output/Output1.csv')

---

### Output 2: Region, Province Name, Type, Population (2015), Population (2010), Annual Population Growth Rat (2010 - 2015), Area (2007 in km2), Density (2015 per km2), City Count, Mun Count, Brgy Count

In [103]:
# VARIABLES
# REGIONS
output2 = pd.DataFrame()

# FOR OUTPUT 3
output3 = pd.DataFrame()
PROVINCES = {}

# FOR OUTPUT 4
MUNICIPALITIES = {}


test2 = {}

In [104]:
for name, link in REGIONS.items():
    right_table = get_specific_element(link, 'table', 'sortable datatable')
    right_table.find_all("tr")

    # VARIABLE INITIALIZATION
    Region = []
    Province = []
    Type = []
    Population_2020 = []
    Population_2015 = []
    APGR_2015_2020 = []
    Area_2013 = []
    Density_2020 = []
    City_count = []
    Mun_count = []
    Brgy_count = []



    # VARIABLE INITIALIZATION for df_output3
    df_output3 = None
    o3_Province = []
    o3_Municipality = []
    o3_Type = []
    o3_Population_2020 = []
    o3_Population_2015 = []
    o3_APGR_2015_2020 = []
    o3_Area_2013 = []
    o3_Density_2020 = []
    o3_Brgy_count = []


    for row in right_table.find_all("tr"):
        cells = row.find_all('td')
        #kunin lahat ng "a" para makuha yung mga region name
        province = row.find_all('a')




        if len(province) > 0:
            if len(cells) > 0:
                Region.append(name)
                Province.append(province[0].find(text=True))
                Type.append(cells[0].find(text=True))
                Population_2020.append(cells[1].find(text=True))
                Population_2015.append(cells[2].find(text=True))
                APGR_2015_2020.append(cells[3].find(text=True))
                Area_2013.append(cells[4].find(text=True))
                Density_2020.append(cells[5].find(text=True))

                # SETUP FOR OUTPUT 3
                if (cells[0].find(text=True)).lower() == 'province':
                    for link in province:
                        _province_url = base_url + link.get('href')
                        # print(link.text)
                        PROVINCES[link.text] = _province_url
                else:
                    # SETUP FOR OUTPUT 4
                    for link in province:
                        _province_url = base_url + link.get('href')
                        # print(link.text)
                        MUNICIPALITIES[(province[0].find(text=True))+'|'+link.text] = _province_url

                    o3_Province.append(name)
                    o3_Municipality.append(province[0].find(text=True))
                    o3_Type.append(cells[0].find(text=True))
                    o3_Population_2020.append(cells[1].find(text=True))
                    o3_Population_2015.append(cells[2].find(text=True))
                    o3_APGR_2015_2020.append(cells[3].find(text=True))
                    o3_Area_2013.append(cells[4].find(text=True))
                    o3_Density_2020.append(cells[5].find(text=True))
                    o3_Brgy_count.append(cells[6].find(text=True))


                # if name != 'NCR':
                if len(cells) > 7:
                    City_count.append(cells[6].find(text=True))
                    Mun_count.append(cells[7].find(text=True))
                    Brgy_count.append(cells[8].find(text=True))


                else:
                    City_count.append('-')
                    Mun_count.append('-')
                    Brgy_count.append(cells[6].find(text=True))




    df_output2 = pd.DataFrame()
    df_output2['Region'] = Region
    df_output2['Province Name'] = Province
    df_output2['Type'] = Type
    df_output2['Population (2020)'] = Population_2020
    df_output2['Population (2015)'] = Population_2015
    df_output2['Annual Population Growth Rate (2015‑2020)'] = APGR_2015_2020
    df_output2['Area (2013), in km2'] = Area_2013
    df_output2['Density (2020), per km2'] = Density_2020

    if len(City_count) > 0:
        df_output2['City count'] = City_count

    if len(Mun_count) > 0:
        df_output2['Mun count'] = Mun_count


    df_output3 = pd.DataFrame()
    df_output3['Province'] = o3_Province
    df_output3['Municipality Name'] = o3_Municipality
    df_output3['Type'] = o3_Type
    df_output3['Population (2020)'] = o3_Population_2020
    df_output3['Population (2015)'] = o3_Population_2015
    df_output3['Annual Population Growth Rate (2015‑2020)'] = o3_APGR_2015_2020
    df_output3['Area (2013), in km2'] = o3_Area_2013
    df_output3['Density (2020), per km2'] = o3_Density_2020

    df_output3['Brgy count'] = o3_Brgy_count

    output3 = output3.append(df_output3)

    

    df_output2['Brgy count'] = Brgy_count
    output2 = output2.append(df_output2)

In [105]:
# PROVINCES
# len(PROVINCES)
# len(MUNICIPALITIES)
# MUNICIPALITIES
# pd.DataFrame.from_dict(enumerate(MUNICIPALITIES))

,Province,Municipality Name,Type,Population (2020),Population (2015),Annual Population Growth Rate (2015‑2020),"Area (2013), in km2","Density (2020), per km2",Brgy count
count,36,36,36,36,36,36,36,36,36
unique,13,36,2,36,36,35,36,36,15
top,NCR,Angeles,city (,"462,928","411,634",0.45%,63.37,"7,305",–
freq,17,1,35,1,1,2,1,1,19


In [106]:
# print(output2)
output2.to_csv(f'../Output/Output2.csv')

In [56]:
# output2.describe()

---

### Output 3: Province, Municipality name, Type, Population (2015), Population (2010), Annual Population Growth Rate (2010 - 2015), Area (2007 in km2), Density (2015 per km2), Brgy Count

In [69]:
# VARIABLES


In [107]:
for i, (name,link) in enumerate(PROVINCES.items()):

    right_table = get_specific_element(link, 'table', 'sortable datatable')
    right_table.find_all("tr")

    # VARIABLE INITIALIZATION
    df_output3 = None
    Province = []
    Municipality = []
    Type = []
    Population_2020 = []
    Population_2015 = []
    APGR_2015_2020 = []
    Area_2013 = []
    Density_2020 = []
    Brgy_count = []

    # CITY / Municipality
    Population_percentage_2020 = []
    Change_2015_2020 = []

    for row in right_table.find_all("tr"):
        cells = row.find_all('td')
        municipality = row.find_all('a')


        if len(municipality) > 0:

            # SETUP FOR OUTPUT 4
            for _link in municipality:
                _municipality_url = base_url + _link.get('href')
                MUNICIPALITIES[name+'|'+_link.text] = _municipality_url

            if len(cells) > 0:

                # PROVINCE
                Province.append(name)
                Municipality.append(municipality[0].find(text=True))
                Type.append(cells[0].find(text=True))
                Population_2020.append(cells[1].find(text=True))
                Population_2015.append(cells[2].find(text=True))
                APGR_2015_2020.append(cells[3].find(text=True))
                Area_2013.append(cells[4].find(text=True))
                Density_2020.append(cells[5].find(text=True))
                Brgy_count.append(cells[6].find(text=True))



    df_output3 = pd.DataFrame()
    df_output3['Province'] = Province
    df_output3['Municipality Name'] = Municipality
    df_output3['Type'] = Type
    df_output3['Population (2020)'] = Population_2020
    df_output3['Population (2015)'] = Population_2015
    df_output3['Annual Population Growth Rate (2015‑2020)'] = APGR_2015_2020
    df_output3['Area (2013), in km2'] = Area_2013
    df_output3['Density (2020), per km2'] = Density_2020

    df_output3['Brgy count'] = Brgy_count

    output3 = output3.append(df_output3)

In [108]:
# MUNICIPALITIES
# len(MUNICIPALITIES)
# pd.DataFrame.from_dict(enumerate(MUNICIPALITIES))

1521

In [109]:

# output3.describe()
# output3.nunique()

,Province,Municipality Name,Type,Population (2020),Population (2015),Annual Population Growth Rate (2015‑2020),"Area (2013), in km2","Density (2020), per km2",Brgy count
count,1521,1521,1521,1521,1521,1521,1521,1521,1521
unique,88,1309,4,1508,1511,492,1481,754,97
top,Cebu,San Isidro,municipality,"36,911","8,573",1.01%,57.46,62,14
freq,50,9,1347,2,2,14,3,10,78


In [131]:

    # PROVINCES
    # REGIONS

    # print(output3)
    output3.to_csv(f'../Output/Output3.csv')
    # output3.describe()


    # for index,(name,link) in enumerate(PROVINCES.items()):
    #     print(PROVINCES_TYPE[index])
    #     print(name)

---

### Output 4: Municipality Name, Barangay Name, Population Percentage (2015), Population (2015), Population (2010), Change (2010 - 2015), Annual Population Growth Rate (2010 - 2015), Postal Code, Coastal/Landlocked, Marine Waterbodies, Coordinates, Estimated Elevation above sea level